In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2025/players.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv
/kaggle/input/nfl-big-data-bowl-2025/games.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv
/kaggle/input/nfl-big-data-bowl-2025/player_play.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv
/kaggle/input/nfl-big-data-bowl-2025/plays.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv


In [2]:
df_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/plays.csv')
df_player_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/player_play.csv')
df_games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/games.csv')
df_players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/players.csv')

In [3]:
df_play.head(1)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,...,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone


In [4]:
df_play['pff_passCoverage'].value_counts()

pff_passCoverage
Cover-3                 4956
Cover-1                 3300
Quarters                2073
Cover-2                 1852
Cover 6-Left             692
Cover-6 Right            690
Cover-3 Seam             636
Cover-0                  605
Red Zone                 537
2-Man                    186
Goal Line                146
Bracket                   75
Cover-1 Double            54
Prevent                   46
Cover-3 Cloud Right       31
Cover-3 Cloud Left        30
Miscellaneous             14
Cover-3 Double Cloud       9
Name: count, dtype: int64

In [5]:
df_play['pff_manZone'].value_counts()

pff_manZone
Zone     10969
Man       4145
Other      818
Name: count, dtype: int64

# Split out coverages, group

Note: we mark cover-3 sub-groups as cover-3, cover-6 as the same, and all else as 'other'

In [6]:
df_condensed = df_play[['gameId','playId','possessionTeam','defensiveTeam']].merge(df_games[['gameId','week']],how='left',on=['gameId'])

In [7]:
all(df_condensed[['gameId','playId']] == df_play[['gameId','playId']])

True

In [8]:
coverage_df = pd.get_dummies(df_play['pff_passCoverage']).astype(int)

In [9]:
len(df_condensed)

16124

In [10]:
cols_c3 = [x for x in coverage_df.columns if 'Cover-3' in x]
cols_c1 = [x for x in coverage_df.columns if 'Cover-1' in x]
cols_c6 = [x for x in coverage_df.columns if (('Cover-6' in x) or ('Cover 6') in x)]

df_condensed['cover_3'] = coverage_df[cols_c3].sum(axis=1).values
df_condensed['cover_6'] = coverage_df[cols_c6].sum(axis=1).values
df_condensed['cover_1'] = coverage_df[cols_c1].sum(axis=1).values

In [11]:
df_covered = pd.concat([df_condensed,coverage_df[['Quarters','Cover-2','Cover-0']]],axis=1)

In [12]:
df_covered['other'] = (~(df_covered.iloc[:,5:].any(axis=1).values)).astype(int)

In [13]:
df_covered.sample(3)

,gameId,playId,possessionTeam,defensiveTeam,week,cover_3,cover_6,cover_1,Quarters,Cover-2,Cover-0,other
13140,2022100907,981,NYJ,MIA,5,0,0,0,0,0,1,0
6950,2022100203,551,ARI,CAR,4,0,0,0,1,0,0,0
8909,2022091500,3114,LAC,KC,2,0,0,0,1,0,0,0


In [14]:
df_covered = pd.concat([df_covered.drop(columns='other'),
                        pd.get_dummies(df_play['pff_manZone']).astype(int)],axis=1)
                                           

In [15]:
df_covered.head(1)

,gameId,playId,possessionTeam,defensiveTeam,week,cover_3,cover_6,cover_1,Quarters,Cover-2,Cover-0,Man,Other,Zone
0,2022102302,2655,CIN,ATL,7,1,0,0,0,0,0,0,0,1


In [16]:
cov_faced = df_covered.drop(columns=['defensiveTeam','gameId',
                                     'playId']).groupby(['possessionTeam','week']).sum().reset_index()
cov_used  = df_covered.drop(columns=['possessionTeam','gameId',
                                     'playId']).groupby(['defensiveTeam','week']).sum().reset_index()

In [17]:
cov_faced.head(1)

,possessionTeam,week,cover_3,cover_6,cover_1,Quarters,Cover-2,Cover-0,Man,Other,Zone
0,ARI,1,8,3,11,16,14,2,13,3,41


### Aggregate cumulative snap counts

Here we get cumulative snap counts (rolling week-to-week) for the coverages each team uses

CF = coverages faced (offense), CU = coverages used  (defense)


In [18]:
df_cum_cf = pd.concat([cov_faced[['possessionTeam','week']], 
                       cov_faced.groupby('possessionTeam').cumsum().drop(columns=['week'])],axis=1)

In [19]:
df_cum_cu = pd.concat([cov_used[['defensiveTeam','week']], 
                       cov_used.groupby('defensiveTeam').cumsum().drop(columns=['week'])],axis=1)

In [20]:
df_cum_cf['week']+=1
df_cum_cf.head(3)

,possessionTeam,week,cover_3,cover_6,cover_1,Quarters,Cover-2,Cover-0,Man,Other,Zone
0,ARI,2,8,3,11,16,14,2,13,3,41
1,ARI,3,21,5,30,23,34,9,39,8,83
2,ARI,4,62,17,37,35,37,9,46,9,151


In [21]:
df_cum_cu['week']+=1
df_cum_cu.head(3)

,defensiveTeam,week,cover_3,cover_6,cover_1,Quarters,Cover-2,Cover-0,Man,Other,Zone
0,ARI,2,21,2,21,7,4,3,26,4,34
1,ARI,3,35,14,32,16,14,4,39,6,79
2,ARI,4,50,21,41,21,16,6,50,9,108


In [22]:
df_cum_cu.columns = ['defensiveTeam','week']+[x+'_def' for x in df_cum_cu.columns[2:]]

In [23]:
df_cum_cu.head(3)

,defensiveTeam,week,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def
0,ARI,2,21,2,21,7,4,3,26,4,34
1,ARI,3,35,14,32,16,14,4,39,6,79
2,ARI,4,50,21,41,21,16,6,50,9,108


In [24]:
df_cum_cf.columns = ['possessionTeam','week']+[x+'_off' for x in df_cum_cu.columns[2:]]

In [25]:
df_cum_cf.head(1)

,possessionTeam,week,cover_3_def_off,cover_6_def_off,cover_1_def_off,Quarters_def_off,Cover-2_def_off,Cover-0_def_off,Man_def_off,Other_def_off,Zone_def_off
0,ARI,2,8,3,11,16,14,2,13,3,41


In [26]:
df_cum_cf.to_csv('coverages_faced.csv')

In [27]:
df_cum_cu.to_csv('coverages_used.csv')